In [1]:
import pandas as pd
import numpy as np
import chardet as chardet
        

In [2]:

with open('Data/BHCF20250331.txt', 'rb') as f:
        raw_data = f.read()
        result = chardet.detect(raw_data)
        encoding = result['encoding']
        print(f"Detected encoding: {encoding}")

Detected encoding: ascii


In [4]:
raw_data = open('Data/BHCF20250331.txt', 'rb').read()
result = chardet.detect(raw_data)
encoding = result['encoding']
print(f"Detected encoding: {encoding}")

Detected encoding: ascii


In [16]:
import glob as glob
all_files = glob.glob('Data/BHCF*.txt')

df_list = []
for file_path in all_files:
    print(file_path)
    raw_data = open(file_path, 'rb').read()
    result = chardet.detect(raw_data)
    encoding = result['encoding']
    df = pd.read_csv(file_path, delimiter = '^', encoding=encoding, engine='python', on_bad_lines='skip')
    df_list.append(df)
    print(encoding,df.shape)
combined_df = pd.concat(df_list, ignore_index=True)

Data/BHCF20250630 3.txt
ascii (3749, 2224)
Data/BHCF20231231.txt
ascii (3813, 2279)
Data/BHCF20230930.txt
ascii (448, 2279)
Data/BHCF20220930.txt
ascii (429, 2328)
Data/BHCF20240630.txt
ascii (3829, 2224)
Data/BHCF20221231.txt
ascii (3919, 2328)
Data/BHCF20240331.txt
ascii (454, 2224)
Data/BHCF20250331.txt
ascii (462, 2224)
Data/BHCF20240930.txt
ascii (446, 2224)
Data/BHCF20220630.txt
ascii (3957, 2327)
Data/BHCF20241231.txt
ISO-8859-1 (3804, 2224)
Data/BHCF20230630.txt
ascii (3887, 2328)
Data/BHCF20250930.txt
ascii (55, 2224)
Data/BHCF20230331.txt
ascii (444, 2328)
Data/BHCF20220331.txt
ascii (432, 2327)


In [17]:
print(combined_df.shape)

(30128, 2328)


In [19]:
dddf = pd.read_csv('/Users/jacksonlipfert/Downloads/MDRM/MDRM_CSV.csv')

In [20]:
#dddf.head()
short_dddf = dddf[['Mnemonic','Item Code','Item Name']]


short_dddf['Full Code'] = short_dddf['Mnemonic'] + short_dddf['Item Code']
important_codes = ['RSSD', 'BHBC', 'BHCA', 'BHCB', 'BHCK', 'BHCM', 'BHCP', 'BHCT', 'BHCW',
       'BHCX', 'BHCY', 'BHDM', 'BHFN', 'BHOD', 'BHPA', 'BHPX', 'BHSP', 'BHSX',
       'BHTX', 'TEXT']
short_dddf = short_dddf[short_dddf['Mnemonic'].isin(important_codes)]


/var/folders/17/1lp6v8rx7hg46v35nrbf4mk00000gn/T/ipykernel_9939/756459364.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  short_dddf['Full Code'] = short_dddf['Mnemonic'] + short_dddf['Item Code']


In [21]:
# create mapper


column_mapper = {}

#Columns that appear across multiple types of companies with the same balance sheet name, need to keep code to disambiguate
special_columns = [
    'TOTAL ASSETS',
    'TOTAL EQUITY',
    'TOTAL EQUITY CAPITAL',
    'NET INCOME',
    'NET INCOME (LOSS)'
    ]

def get_name_from_full_code(fullcode, df):
    '''get column name from full code found in data dictionary'''
    column = df['Item Name'][df['Full Code'] == fullcode]
    col_name = column.iloc[0]
    return col_name


#Loop over short data dictionary and populate column mapper with name
for code in short_dddf['Full Code']:
    column_mapper[code] = get_name_from_full_code(code,short_dddf)


def disambiguate_column_mapper(column_mapper, special_columns):
    '''Loop over column mapper and replace ambigious columns (found in special_columns)'''

    for key,value in column_mapper.items():
        if value in special_columns:
            column_mapper[key] = f'{value}: {key}' 
    return column_mapper

#Final column mapper
disambig_column_mapper = disambiguate_column_mapper(column_mapper, special_columns)

#check to make sure columns have been renamed
column_name_in_question = 'NET INCOME'
for key,value in disambig_column_mapper.items():
    if column_name_in_question in str(value):
        print(value)
    


    

NET INCOME (LOSS): BHBC4340
NET IMPACT ON NET INCOME (BEFORE TAXES) OF THE SALE OF OTHER ASSETS WITH RECOURSE
NET IMPACT ON NET INCOME (BEFORE TAXES) OF LOAN ASSETS THAT WERE SOLD WITH RECOURSE
NET IMPACT ON NET INCOME (BEFORE TAXES) OF DEFEASED DEBT
NET INCOME (LOSS): BHCK4340
NET INCOME BEFORE INCOME TAXES; EXTRAORDINARY ITEMS; AND OTHER ADJUSTMENTS ON A FULLY TAXABLE EQUIVALENT BASIS
NET INCOME FOR ALL INSURANCE-RELATED ACTIVITIES
NET INCOME: BHCKC246
NET INCOME: BHCKC250
LESS: NET INCOME (LOSS) ATTRIBUTABLE TO NONCONTROLLING (MINORITY) INTERESTS
NET INCOME (LOSS) ATTRIBUTABLE TO BANK AND NONCONTROLLING MINORITY INTERESTS
NET INCOME FROM SERVICE CORPORATION/SUBSIDIARIES (SEE NOTE BELOW)
CASH FLOWS FROM OPERATING ACTIVITIES: ADJUSTMENTS TO RECONCILE NET INCOME TO NET CASH PROVIDED BY OPERATING ACTIVITIES - PROVISION FOR DEFERRED INCOME TAXES
CASH FLOWS FROM OPERATING ACTIVITIES: ADJUSTMENTS TO RECONCILE NET INCOME TO NET CASH PROVIDED BY OPERATING ACTIVITIES - (GAIN) OR LOSS ON SALES

In [22]:
renamed_combined_df = combined_df.rename(columns=disambig_column_mapper)
print(renamed_combined_df.shape)

(30128, 2328)


In [25]:
search_string = 'REPORT'
matching_columns = renamed_combined_df.columns[renamed_combined_df.columns.str.contains(search_string)]
print(matching_columns)

Index(['REPORTING DATE (CC;YR;MO;DA)',
       'LESS: NET UNREALIZED GAINS (LOSSES) ON AVAILABLE-FOR-SALE DEBT SECURITIES (IF A GAIN; REPORT AS A POSITIVE VALUE; IF A LOSS; REPORT AS A NEGATIVE VALUE)',
       'ACCUMULATED NET GAINS (LOSSES)ON CASH-FLOW HEDGES (IF A GAIN; REPORT AS A POSITIVE VALUE; IF A LOSS; REPORT AS A NEGATIVE VALUE)',
       'LESS: AMOUNTS RECORDED IN AOCI ATTRIBUTED TO DEFINED BENEFIT POSTRETIREMENT PLANS RESULTING FROM THE INITIAL AND SUBSEQUENT APPLICATION OF THE RELEVANT GAAP STANDARDS THAT PERTAIN TO SUCH PLANS (IF A GAIN; REPORT AS A POSITIVE VALUE; IF A LOSS; REPORT AS A NEGATIVE VALUE)',
       'LESS: NET UNREALIZED GAINS (LOSSES) ON HELD-TO-MATURITY SECURITIES THAT ARE INCLUDED IN AOCI (IF A GAIN; REPORT AS A POSITIVE VALUE; IF A LOSS; REPORT AS A NEGATIVE VALUE)',
       'EQUITY CAPITAL; MOST RECENTLY REPORTED FOR THE END OF PREVIOUS CALENDAR YEAR (I.E.; AFTER ADJUSTMENTS FROM AMENDED REPORTS OF INCOME)',
       'LONG-TERM DEBT REPORTED IN SCHEDULE HC; IT

In [27]:
renamed_combined_df[['ENTITY SHORT NAME','REPORTING DATE (CC;YR;MO;DA)']]

,ENTITY SHORT NAME,REPORTING DATE (CC;YR;MO;DA)
0,HSBC USA,20250630
1,SOUTHERN NAT CORP,20250630
2,WOODTRUST FC,20250630
3,AMALGAMATED INV CO,20250630
4,BANK MGMT,20250630
...,...,...
30123,EB ACQ CO II LLC,20220331
30124,CSBH LLC,20220331
30125,CARTER BSHRS,20220331
30126,AMALGAMATED FNCL CORP,20220331
